In [1]:
#baseline
from sklearn.linear_model import Lasso

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [375]:
import pandas as pd
import seaborn as sns
import numpy as np

In [395]:
X = pd.read_pickle('X_1000.pkl')

In [396]:
y = pd.read_pickle('y_1000.pkl')

In [397]:
X.columns = X.columns.astype(str)

In [398]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

In [399]:
y_true_mean = y.mean()

In [420]:
from sklearn.linear_model import QuantileRegressor
from sklearn.model_selection import cross_val_score
solver = "highs"
regression = QuantileRegressor(quantile=0.5, alpha=0.057, solver=solver)
y_pred = regression.fit(X_train, y_train).predict(X_test)
np.mean(cross_val_score(regression, X_train, y_train, cv=5, scoring='r2'))

0.6818359309596093

In [258]:
import quantile_scoring as scr
alpha_test=0.2
scr.interval_score(obs_test,alpha_test,q_left=predictions[0.025],q_right=predictions[0.975])

(array([16.36763519,  5.44255296,  3.20084101,  6.17468916,  5.47026593,
         4.02756826,  7.18473576, 23.54181868,  4.48211095,  6.10996279,
         4.50872845,  5.91175358,  6.71291957,  3.95299262,  5.49449979,
         5.13884938,  5.48629481,  4.91874855, 16.87385564,  5.73136669,
         6.4248452 ,  3.95001878,  2.65198059,  4.83162313,  4.20553012,
         1.36992991,  4.21675034, 30.44045704,  2.32935924,  6.71370032,
         5.33252324,  5.21905582,  5.06212204,  8.36688149, 10.35913277,
         3.94259454,  5.27205672, 18.10004325,  7.91077729,  2.62082992,
         3.71663098,  9.63660984,  4.15811864, 24.75795115,  7.1591745 ,
        11.12075974,  6.15737428,  8.79930063,  2.03573685,  6.62712372,
         7.10883056,  5.93997397,  5.18786375,  2.04480536,  3.99426588,
         6.90151145,  5.19263103,  3.92871736,  2.69329741,  4.96111944,
         7.24199651,  5.61121439,  4.25481782,  5.13884938,  5.84166973,
         4.52205681,  7.28010755,  3.41159783,  5.1

In [335]:
X_2 = df_merged.drop(['PTS', 'TEAM_ID', 'OFF_RATING', 'EFG_PCT'], axis=1)
y_2 = df_merged['PTS']

In [336]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X_2, y_2, random_state=0, test_size=0.2)

In [337]:
mx_scaler2 = MinMaxScaler()
X2_train = mx_scaler.fit_transform(X2_train)
X2_test = mx_scaler.transform(X2_test)

In [338]:
y2_true_mean = y_2.mean()

In [345]:
predictions2 = {}
scores2 = {}
out_bounds_predictions = np.zeros_like(y2_true_mean, dtype=np.bool_)
for quantile in quantiles:
    qr2 = QuantileRegressor(quantile=quantile, alpha=0.005, solver=solver)
    y2_pred = qr.fit(X2_train, y2_train).predict(X2_test)
    predictions2[quantile] = y2_pred
    scores2[quantile] = cross_val_score(qr2, X2_train, y2_train, cv=5, scoring='r2')

    if quantile == min(quantiles):
        out_bounds_predictions = np.logical_or(
            out_bounds_predictions, y2_pred >= y2_test
        )
    elif quantile == max(quantiles):
        out_bounds_predictions = np.logical_or(
            out_bounds_predictions, y2_pred <= y2_test
        )

In [346]:
print(scores2[0.025])
print(scores2[0.5])
print(scores2[0.975])

[-3.28337394 -3.79601085 -3.50640535 -3.37807261 -3.08342673]
[0.99762027 0.99778954 0.99588966 0.99770141 0.99662594]
[-3.61416404 -3.66402199 -3.81771701 -5.11401248 -3.98919511]
